In [11]:
from torch.utils.data import DataLoader
from resnet import ResNet
from sklearn.model_selection import train_test_split
import numpy as np
from custom_dataset import FinalDataset
from SubSet import SignalDataset
import torch
from torch import nn

In [ ]:
file_path= "D:\\thesis_data\\filteredData.hdf5"

In [ ]:
dataset = FinalDataset(file_path)
for i in range(len(dataset)):
    dataset.plotSignal(i)

In [ ]:
total_size = len(dataset)
indices = np.arange(total_size)
np.random.shuffle(indices)
train_indices, test_indices = train_test_split(indices, test_size=0.2)

train_dataset = FinalDataset(file_path, indices=train_indices)
test_dataset = FinalDataset(file_path, indices=test_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle =True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
model = ResNet(12,24)

In [ ]:
model.train()

In [ ]:
for id, (signal, label) in enumerate(train_loader):
    output = model(signal)
    print(output)
    break


In [12]:
signal_dataset = SignalDataset(file_path)
signal_dataloader = DataLoader(signal_dataset, batch_size=32, shuffle=True)

In [ ]:
from customDataloader import CustomDataloader

In [1]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

for epoch in range(10):
    for id, (signal, label) in enumerate(signal_dataloader):
        output = model(signal)
        output = output.squeeze()
        loss = loss_fn(output, label.float())
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")


NameError: name 'torch' is not defined